In [1]:
# --- Imports ---
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc
from dash.dependencies import Input, Output
from typing import Union, Tuple
import datetime

# --- Core Logic: Growth Rate Calculation ---

def calculate_growth_rate(
    start_value: Union[int, float],
    end_value: Union[int, float]
) -> float:
    """
    Calculates the percentage growth rate between two values.

    Formula: ((end_value - start_value) / start_value) * 100

    Args:
        start_value: The numeric value at the start of the period (denominator).
        end_value: The numeric value at the end of the period (numerator).

    Returns:
        The growth rate as a percentage (float).

    Raises:
        ValueError: If the start_value is zero.
    """
    if start_value == 0:
        # Prevent division by zero error unless both start and end are zero.
        if end_value == 0:
            return 0.0
        raise ValueError("Cannot calculate percentage growth from a zero baseline.")

    growth_rate = ((end_value - start_value) / start_value) * 100
    return growth_rate

In [2]:
# --- Data Simulation ---
# Generate 12 monthly dates ending today
dates = pd.date_range(end=datetime.date.today(), periods=12, freq='M')

# Simulate increasing monthly sales figures
sales_data = {
    'Date': dates,
    # Example sales values for the 12 periods
    'Sales_USD': [10000, 10500, 11000, 10800, 12000, 12500, 13000, 13500, 14000, 15000, 16500, 18000]
}
df = pd.DataFrame(sales_data)

# --- Dash Application Setup ---
# Initialize the Dash application
app = Dash(__name__)

C:\Users\User\AppData\Local\Temp\ipykernel_14788\541148149.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(end=datetime.date.today(), periods=12, freq='M')


In [3]:
# --- Dashboard Layout Definition ---
app.layout = html.Div(
    style={'padding': '20px', 'backgroundColor': '#f9f9f9'},
    children=[
        # Main Title
        html.H1(
            "📈 Sales Growth Rate Dashboard",
            style={'textAlign': 'center', 'color': '#333'}
        ),
        html.Hr(style={'borderColor': '#ccc'}),

        # --- User Input Control Section ---
        html.Div(
            style={'display': 'flex', 'justifyContent': 'center', 'gap': '50px', 'marginBottom': '30px', 'padding': '15px', 'backgroundColor': '#fff', 'borderRadius': '8px', 'boxShadow': '0 2px 4px rgba(0,0,0,0.1)'},
            children=[
                # Start Month Selector Dropdown
                html.Div(children=[
                    html.Label('Select Start Month:', style={'fontWeight': 'bold', 'color': '#555'}),
                    dcc.Dropdown(
                        id='start-month-dropdown',
                        # Options are formatted as YYYY-MM for display, but full date for value
                        options=[{'label': i.strftime('%Y-%m'), 'value': i.strftime('%Y-%m-%d')} for i in df['Date']],
                        value=df['Date'].iloc[0].strftime('%Y-%m-%d'),  # Default to the first date
                        clearable=False,
                        style={'width': '200px'}
                    )
                ]),

                # End Month Selector Dropdown
                html.Div(children=[
                    html.Label('Select End Month:', style={'fontWeight': 'bold', 'color': '#555'}),
                    dcc.Dropdown(
                        id='end-month-dropdown',
                        options=[{'label': i.strftime('%Y-%m'), 'value': i.strftime('%Y-%m-%d')} for i in df['Date']],
                        value=df['Date'].iloc[-1].strftime('%Y-%m-%d'), # Default to the last date
                        clearable=False,
                        style={'width': '200px'}
                    )
                ]),
            ]
        ),

        # --- Key Performance Indicator (KPI) Display ---
        html.Div(
            id='kpi-output',
            style={'textAlign': 'center', 'marginBottom': '40px', 'padding': '20px', 'backgroundColor': '#e8f0fe', 'borderRadius': '8px', 'borderLeft': '5px solid #4a90e2'},
            children=[
                html.H2("Calculated Sales Growth Rate:", style={'color': '#1f4e79'}),
                # Target for the calculated growth rate value
                html.H3(id='growth-rate-value', style={'fontSize': '3em', 'fontWeight': 'bold', 'color': '#4a90e2'})
            ]
        ),

        # --- Sales Trend Chart ---
        html.Div(
            style={'padding': '15px', 'backgroundColor': '#fff', 'borderRadius': '8px', 'boxShadow': '0 2px 4px rgba(0,0,0,0.1)'},
            children=[
                html.H3("Monthly Sales Trend", style={'textAlign': 'center', 'color': '#333'}),
                # Target for the Plotly line graph
                dcc.Graph(id='sales-trend-chart')
            ]
        )
    ]
)

In [4]:
# --- Callbacks for Interactivity ---

@app.callback(
    # Outputs: Update the growth rate text and the chart figure
    [Output('growth-rate-value', 'children'),
     Output('sales-trend-chart', 'figure')],
    # Inputs: Listen for changes in the start and end month dropdowns
    [Input('start-month-dropdown', 'value'),
     Input('end-month-dropdown', 'value')]
)
def update_dashboard(start_date_str: str, end_date_str: str):
    """
    Updates the growth rate KPI and the sales trend chart based on user selection.
    This function executes automatically whenever the input values change.
    """
    # 1. Date and Input Conversion/Validation
    start_date = datetime.datetime.strptime(start_date_str, '%Y-%m-%d').date()
    end_date = datetime.datetime.strptime(end_date_str, '%Y-%m-%d').date()

    # Check for invalid chronological order
    if end_date < start_date:
        return (
            "⚠️ Start date must be before end date.",
            # Return a simple figure when input is invalid
            px.line(df, x='Date', y='Sales_USD', title='Sales Trend (Invalid Range Selected)')
        )

    # 2. Extract Values
    # Filter the DataFrame to get the sales value for the exact start date
    start_value = df[df['Date'].dt.date == start_date]['Sales_USD'].iloc[0]
    
    # Filter the DataFrame to get the sales value for the exact end date
    end_value = df[df['Date'].dt.date == end_date]['Sales_USD'].iloc[0]

    # 3. Calculate Growth Rate and Format KPI
    try:
        rate = calculate_growth_rate(start_value, end_value)
        # Format rate to show sign (+ or -) and two decimal places
        kpi_text = f"{rate:+.2f}%" 
    except ValueError as e:
        # Display the error message if calculation fails (e.g., zero baseline)
        kpi_text = f"Error: {e}"
        rate = 0.0 # Set rate to 0 for chart logic if an error occurs

    # 4. Prepare and Update Chart
    
    # Filter data for the selected date range (inclusive)
    filtered_df = df[
        (df['Date'].dt.date >= start_date) & 
        (df['Date'].dt.date <= end_date)
    ]
    
    # Create Plotly line chart
    fig = px.line(
        filtered_df, 
        x='Date', 
        y='Sales_USD', 
        title=f'Sales Trend from {start_date} to {end_date}',
        markers=True, 
        line_shape='linear'
    )
    
    # Customize line color based on growth/decline
    line_color = '#28a745' if rate >= 0 else '#dc3545'
    fig.update_traces(line=dict(color=line_color, width=3))
    
    # Add annotations for clear start/end point identification
    fig.add_annotation(
        x=start_date, y=start_value, text=f"Start: ${start_value:,.0f}", showarrow=True, arrowhead=1, yshift=10
    )
    fig.add_annotation(
        x=end_date, y=end_value, text=f"End: ${end_value:,.0f}", showarrow=True, arrowhead=1, yshift=-15
    )
    
    fig.update_layout(
        xaxis_title="Date",
        yaxis_title="Sales (USD)",
        hovermode="x unified"
    )

    return kpi_text, fig

In [5]:
# --- Run the Application ---

if __name__ == '__main__':
    # When this script is run, the Dash server starts.
    print("Dash application running at http://127.0.0.1:8050/")
    # debug=True allows for automatic code reloading upon changes
    app.run_server(debug=True)

# Instructions to Run:
# 1. Save the code as 'sales_dashboard.py'
# 2. Ensure you have installed pandas, dash, and plotly: pip install pandas dash plotly
# 3. Run from terminal: python sales_dashboard.py
# 4. Open your web browser and navigate to the address shown above (typically http://127.0.0.1:8050/)

Dash application running at http://127.0.0.1:8050/


ObsoleteAttributeException: app.run_server has been replaced by app.run